# Derivatives of Supernatants from our freezers, after corrections

## General informations

In [3]:
import pandas as pd
import re
from IPython.display import display, Markdown, HTML
from file_toolkit import *

pd.set_option('display.max_rows', 96)
pd.set_option('display.max_columns', 96)

filepath = "/Volumes/LabExMI/Users/Nolwenn/FreezerPro/DataToImport/"
filename = "Supernatants_Derivatives_F1F2_20170118_tubescorrected.csv"
df = pd.read_csv(filepath+filename, sep=";")
display(Markdown("**%d** tubes in our derivatives file." % len(df)))
display(Markdown("* **%d** tubes for Fraction 1." % \
                 len(df.loc[df["Sample Type"] == "Fraction1"])))
display(Markdown("* **%d** tubes for Fraction 2." % \
                 len(df.loc[df["Sample Type"] == "Fraction2"])))
display(Markdown("* **%d** tubes are not assigned to a fraction." % \
                 len(df.loc[df["Sample Type"].isnull()])))
show_columns(df)

**20000** tubes in our derivatives file.

* **10000** tubes for Fraction 1.

* **10000** tubes for Fraction 2.

* **0** tubes are not assigned to a fraction.

List of the *33* columns:

1. ParentID;
1. Name;
1. BARCODE;
1. Position;
1. Volume;
1. Freezer;
1. Freezer_Descr;
1. Level1;
1. Level1_Descr;
1. Level2;
1. Level2_Descr;
1. Level3;
1. Level3_Descr;
1. BoxType;
1. Box;
1. Box_Descr;
1. ThermoBoxBarcode;
1. BOX_BARCODE;
1. CreationDate;
1. UpdateDate;
1. AliquotID;
1. DonorID;
1. StimulusID;
1. StimulusName;
1. VisitID;
1. ThawCycle;
1. Sample Source;
1. Description;
1. BatchID;
1. Sample Type;
1. ShelfBarcode;
1. RackBarcode;
1. DrawerBarcode.

We have **20.000** tubes as expected, and **10.000** tubes per Fraction. We want to know how many tube per donor we have.

## Checking tubes

In [4]:
display(Markdown("Final output file has **%d unique StimulusID**" % \
                 len(df["StimulusID"].unique())))
countstimperdonor = pd.DataFrame(df.groupby("DonorID")["StimulusID"].count())
display(Markdown("List of **%d** count of tubes per donor:" % \
                 len(countstimperdonor["StimulusID"].unique())))
display(Markdown(";\n".join(["* "+str(int(stim))+" tubes per donor" for stim \
                             in sorted(countstimperdonor["StimulusID"].unique())])+"."))

Final output file has **10 unique StimulusID**

List of **1** count of tubes per donor:

* 20 tubes per donor.

We were expecting **10 unique** StimulusID, Céline and Bruno treatet 10 Stimulus. We have an extra stimulus.

We want to know if at least one donor has less than 20 tubes associated, and, if possible, which tube is missing.

In [5]:
donorindexes = countstimperdonor[countstimperdonor["StimulusID"] < 20].\
index.values
if len(donorindexes) > 0:
    display(Markdown("List of **%d** unique DonorID:" % len(donorindexes)))
    display(Markdown(";\n".join([" - "+str(int(donor)) for donor \
                                 in sorted(donorindexes)])+"."))
else:
    display(Markdown("All donor are assigned to at least 20 StimulusID."))
    display(Markdown("**%d** unique DonorID has more than 20 StimulusID." % \
                     len(countstimperdonor[countstimperdonor["StimulusID"] > 20].\
                         index.values)))

All donor are assigned to at least 20 StimulusID.

**0** unique DonorID has more than 20 StimulusID.

We also want to known how many DonorID each StimulusID count:

In [6]:
compare_two_columns(df, "StimulusID", "DonorID")
display(Markdown("**%d** unique DonorID." % len(df["DonorID"].unique())))

,StimulusID,Nb_DonorID
0,11.0,2000
1,17.0,2000
2,18.0,2000
3,23.0,2000
4,24.0,2000
5,27.0,2000
6,32.0,2000
7,35.0,2000
8,37.0,2000
9,39.0,2000


**1000** unique DonorID.

Just to be sure, we need to know how many unique barcode each StimulusID count:

In [7]:
compare_two_columns(df, "StimulusID", "BARCODE")
display(Markdown("**%d** unique BARCODE." % len(df["BARCODE"].unique())))
display(Markdown("**%d** not unique BARCODE:" % \
                 len(df.loc[df.duplicated(["BARCODE"])])))
if len(df.loc[df.duplicated(["BARCODE"])]) > 0:
    barcodelist = df.loc[df.duplicated(["BARCODE"]), \
                         "BARCODE"].tolist()
    display(df.loc[df["BARCODE"].isin(barcodelist), \
                   ["ParentID", "Position", "DonorID", "AliquotID", "StimulusID"]])

,StimulusID,Nb_BARCODE
0,11.0,2000
1,17.0,2000
2,18.0,2000
3,23.0,2000
4,24.0,2000
5,27.0,2000
6,32.0,2000
7,35.0,2000
8,37.0,2000
9,39.0,2000


**20000** unique BARCODE.

**0** not unique BARCODE:

Why do the BARCODE *3260627121* is duplicated? Need to recheck.

For donor *819*, the problem is normally corrected. In fact, this donor is supposed to be assigned to stimulus 23 but it seems that stimulus were mixed for this donor:
* in box 23, tube donor 819 found should have been assigned for box 24
* in box 24, tube donor 819 found should have been assigned for box 17
* in box 17, tube donor 819 found should have been assigned for box 18
* in box 18, tube donor 819 found should have been assigned for box 19

Nolwenn manually changed the wrong ParentID and StimulusID. The following analysis will track if an error still occurs.

For stimulus 23, we want to know which are the missing donors:

In [8]:
donorlist = df["DonorID"].unique()
donorliststim23 = df.loc[df["StimulusID"] == 23.0, "DonorID"].unique()
donornotfound = list(set(donorlist) - set(donorliststim23))
if len(donornotfound) > 0:
    display(Markdown("List of **%d** donors not found for StimulusID 23:" % \
                     len(donornotfound)))
    display(Markdown(";\n".join(["* "+str(int(donor)) for donor \
                                 in sorted(donornotfound)])))
else:
    display(Markdown("List of **%d** donors not found for StimulusID 23." % \
                     len(donornotfound)))

List of **0** donors not found for StimulusID 23.

## Checking boxes

Know we want to be sure that we have one stimulus per box:

In [9]:
display(Markdown("There is **%d unique boxes** in the whole file." % \
                 len(df["Box"].unique())))

display(Markdown("**Look on Fraction 1:**"))
display(Markdown("There is **%d unique boxes** in the file for Fraction 1." % \
                 len(df.loc[df["Sample Type"] == "Fraction1", "Box"].unique())))
countboxstim = pd.DataFrame(df.groupby("Box")["StimulusID"].nunique())
countboxstim.loc[:, "Box"] = countboxstim.index.get_values()
countboxstim.reset_index(drop = True, inplace = True)
if len(countboxstim[countboxstim["StimulusID"] > 1]) > 0:
    display(table_wo_index(countboxstim.loc[(countboxstim["StimulusID"] > 1) & \
                                            (countboxstim["Box"].str.contains("F1")),\
                             ["Box", "StimulusID"]]))
else:
    display(Markdown("None box contains more than one stimulus."))

display(Markdown("**Look on Fraction 2:**"))
display(Markdown("There is **%d unique boxes** in the file for Fraction 2." % \
                 len(df.loc[df["Sample Type"] == "Fraction2", "Box"].unique())))
countboxstim = pd.DataFrame(df.groupby("Box")["StimulusID"].nunique())
countboxstim.loc[:, "Box"] = countboxstim.index.get_values()
countboxstim.reset_index(drop = True, inplace = True)
if len(countboxstim[countboxstim["StimulusID"] > 1]) > 0:
    display(table_wo_index(countboxstim.loc[(countboxstim["StimulusID"] > 1) & \
                                            (countboxstim["Box"].str.contains("F2")),\
                             ["Box", "StimulusID"]]))
else:
    display(Markdown("None box contains more than one stimulus."))

There is **220 unique boxes** in the whole file.

**Look on Fraction 1:**

There is **110 unique boxes** in the file for Fraction 1.

None box contains more than one stimulus.

**Look on Fraction 2:**

There is **110 unique boxes** in the file for Fraction 2.

None box contains more than one stimulus.

The boxes impacted shown in previous report seems to have been succesfully corrected.

We also want to be sure that the box name reproduce the same correction if we look with the Thermo Fisher box barcode:

In [10]:
display(Markdown("There is a total of **%d unique boxes** in the file." % \
                 len(df["ThermoBoxBarcode"].unique())))
display(Markdown("There is **%d unique boxes** in Fraction 1." % \
                 len(df.loc[df["Sample Type"] == "Fraction1", \
                            "ThermoBoxBarcode"].unique())))
display(Markdown("There is **%d unique boxes** in Fraction 2." % \
                 len(df.loc[df["Sample Type"] == "Fraction2", 
                            "ThermoBoxBarcode"].unique())))

countthermoboxstim = compare_two_columns(df, "ThermoBoxBarcode", "StimulusID", \
                                        method="nunique", show=False)
if len(countthermoboxstim.loc[(countthermoboxstim["StimulusID"] > 1)]> 0):
    display(countthermoboxstim.loc[(countthermoboxstim["StimulusID"] > 1), \
                                   ["ThermoBoxBarcode", "StimulusID"]])
else:
    display(Markdown("None box contains more than one stimulus."))

There is a total of **220 unique boxes** in the file.

There is **110 unique boxes** in Fraction 1.

There is **110 unique boxes** in Fraction 2.

None box contains more than one stimulus.

We have the same number of ThermoBoxBarcode as of BoxBarcode.

For each ThermoBoxBarcode column, which are those with more than one Box column associated?

In [11]:
countboxperthermobox = compare_two_columns(df, "Box", "ThermoBoxBarcode", \
                                          method="nunique", show=False)

display(Markdown("**%d** boxes are not assigned to a ThermoBoxBarcode" % \
                 len(countboxperthermobox.loc[countboxperthermobox["ThermoBoxBarcode"] < 1, \
                                              ["Box", "ThermoBoxBarcode"]])))
display(Markdown("**%d** boxes are assigned to more than one ThermoBoxBarcode" % \
                 len(countboxperthermobox.loc[countboxperthermobox["ThermoBoxBarcode"] > 1, \
                                              ["Box", "ThermoBoxBarcode"]])))
display(Markdown("**%d** boxes are assigned to a ThermoBoxBarcode" % \
                 len(countboxperthermobox.loc[countboxperthermobox["ThermoBoxBarcode"] == 1, \
                                              ["Box", "ThermoBoxBarcode"]])))

if len(countboxperthermobox.loc[countboxperthermobox["ThermoBoxBarcode"] != 1, \
                                ["Box", "ThermoBoxBarcode"]]) > 0:
    display(countboxperthermobox.loc[countboxperthermobox["ThermoBoxBarcode"] != 1, \
                                     ["Box", "ThermoBoxBarcode"]])

**0** boxes are not assigned to a ThermoBoxBarcode

**0** boxes are assigned to more than one ThermoBoxBarcode

**220** boxes are assigned to a ThermoBoxBarcode

No more than one Box column is associated to a ThermoBoxBarcode, as expected. Do we have tubes that are not assigned to ThermoBoxBarcode column?

In [12]:
boxes = countboxperthermobox.loc[countboxperthermobox["ThermoBoxBarcode"] != 1]\
["Box"].values
display(Markdown("**%d** tubes in boxes not assigned to ThermoBoxbarcode." % \
      len(df.loc[df["Box"].isin(boxes), ["Box", "ThermoBoxBarcode", "DonorID"]])))
if len(df.loc[df["Box"].isin(boxes), ["Box", "ThermoBoxBarcode", "DonorID"]]) > 0:
    display(df.loc[df["Box"].isin(boxes), ["Box", "ThermoBoxBarcode", "DonorID"]])

**0** tubes in boxes not assigned to ThermoBoxbarcode.

## Checking excluded donors

We also want to check if none of the excluded donors are in our data:

In [13]:
excludeddonors = [96, 104, 122, 167, 178, 219, 268, 279, 303, 308, 534, 701]
df["DonorID"] = df["DonorID"].astype(int)
display(Markdown("**%d** donor found." % len(df.loc[df["DonorID"].isin(excludeddonors), \
                                                    "DonorID"].unique())))
if len(df.loc[df["DonorID"].isin(excludeddonors), "DonorID"].unique()) > 0:
    display(Markdown("The excluded donor found are:"))
    display(Markdown(";\n".join(["* "+str(donor) for donor \
                                 in df.loc[df["DonorID"].isin(excludeddonors), \
                                           "DonorID"].unique()])))

**0** donor found.

We have none of the excluded donors in the final output of our data.

## Checking StimulusID

We are supposed to have 11 boxes per stimulus and fraction, how many boxes do we effectively have per StimulusID?

In [14]:
display(Markdown("**Look on Fraction 1:**"))
df_frac1 = df.loc[df["Sample Type"] == "Fraction1"]
boxperstimfrac1 = compare_two_columns(df_frac1, "StimulusID", "Box")

display(Markdown("**Look on Fraction 2:**"))
df_frac2 = df.loc[df["Sample Type"] == "Fraction2"]
boxperstimfrac2 = compare_two_columns(df_frac2, "StimulusID", "Box")

**Look on Fraction 1:**

,StimulusID,Nb_Box
0,11.0,1000
1,17.0,1000
2,18.0,1000
3,23.0,1000
4,24.0,1000
5,27.0,1000
6,32.0,1000
7,35.0,1000
8,37.0,1000
9,39.0,1000


**Look on Fraction 2:**

,StimulusID,Nb_Box
0,11.0,1000
1,17.0,1000
2,18.0,1000
3,23.0,1000
4,24.0,1000
5,27.0,1000
6,32.0,1000
7,35.0,1000
8,37.0,1000
9,39.0,1000


We have 11 boxes per stimulus, as expected. The boxes in StimulusID 17, 18, and 24 generated errors on a previous report, now they are in accordance with expected data:

In [15]:
display(df.loc[(df["StimulusID"].isin([17.0, 18.0, 19.0, 24.0])) & \
               (df["DonorID"] == 819), \
               ["Box", "StimulusID", "DonorID"]])

,Box,StimulusID,DonorID
3674,MIC_Plasma_S17_V1_A1_F1_D801-896,17.0,819
3675,MIC_Plasma_S17_V1_A1_F2_D801-896,17.0,819
5550,MIC_Plasma_S18_V1_A1_F1_D801-896,18.0,819
5551,MIC_Plasma_S18_V1_A1_F2_D801-896,18.0,819
9550,MIC_Plasma_S24_V1_A1_F1_D801-896,24.0,819
9551,MIC_Plasma_S24_V1_A1_F2_D801-896,24.0,819


## Checking common fields

We have 2 fractions, but each fraction have to contains common informations:

* ParentID;
* Position;
* CreationDate;
* UpdateDate;
* AliquotID;
* BoxType;
* VisitID;
* ThawCycle;
* Volume;
* Sample Source;
* BatchID;
* Sample Type.

We expect to have same number of lines for each field, for each fraction:

In [16]:
display(Markdown("### Look on Fraction 1"))

display(Markdown("#### ParentID field"))
display(Markdown("* %d empty ParentID\n* %d unique ParentID" % \
                 (len(df_frac1[df_frac1["ParentID"].isnull()]), \
                  len(df_frac1["ParentID"].unique()))))

display(Markdown("#### Position field"))
tubesperposition1 = pd.DataFrame(df_frac1.groupby("Position")["Position"].count())
tubesperposition1.loc[:, "Tubes"] = tubesperposition1["Position"]
tubesperposition1.loc[:, "Position"] = tubesperposition1.index
tubesperposition1.reset_index(drop=True, inplace=True)
display(tubesperposition1)
display(Markdown("**%d** tubes for Position." % tubesperposition1["Tubes"].sum()))

display(Markdown("#### CreationDate field"))
tubespercreation1 = pd.DataFrame(df_frac1.groupby("CreationDate")["CreationDate"].count())
tubespercreation1.loc[:, "Tubes"] = tubespercreation1["CreationDate"]
tubespercreation1.loc[:, "CreationDate"] = tubespercreation1.index
tubespercreation1.reset_index(drop=True, inplace=True)
display(tubespercreation1)
display(Markdown("**%d** tubes for CreationDate." % tubespercreation1["Tubes"].sum()))

display(Markdown("#### UpdateDate field"))
tubesperupdate1 = pd.DataFrame(df_frac1.groupby("UpdateDate")["UpdateDate"].count())
tubesperupdate1.loc[:, "Tubes"] = tubesperupdate1["UpdateDate"]
tubesperupdate1.loc[:, "UpdateDate"] = tubesperupdate1.index
tubesperupdate1.reset_index(drop=True, inplace=True)
display(tubesperupdate1)
display(Markdown("**%d** tubes for UpdateDate." % tubesperupdate1["Tubes"].sum()))

display(Markdown("#### AliquotID field"))
tubesperaliquotid1 = pd.DataFrame(df_frac1.groupby("AliquotID")["AliquotID"].count())
tubesperaliquotid1.loc[:, "Tubes"] = tubesperaliquotid1["AliquotID"]
tubesperaliquotid1.loc[:, "AliquotID"] = tubesperaliquotid1.index
tubesperaliquotid1.reset_index(drop=True, inplace=True)
display(tubesperaliquotid1)
display(Markdown("**%d** tubes for AliquotID." % tubesperaliquotid1["Tubes"].sum()))

display(Markdown("#### BoxType field"))
tubesperboxtype1 = pd.DataFrame(df_frac1.groupby("BoxType")["BoxType"].count())
tubesperboxtype1.loc[:, "Tubes"] = tubesperboxtype1["BoxType"]
tubesperboxtype1.loc[:, "BoxType"] = tubesperboxtype1.index
tubesperboxtype1.reset_index(drop=True, inplace=True)
display(tubesperboxtype1)
display(Markdown("**%d** tubes for BoxType." % tubesperboxtype1["Tubes"].sum()))

display(Markdown("#### VisitID field"))
tubespervisitid1 = pd.DataFrame(df_frac1.groupby("VisitID")["VisitID"].count())
tubespervisitid1.loc[:, "Tubes"] = tubespervisitid1["VisitID"]
tubespervisitid1.loc[:, "VisitID"] = tubespervisitid1.index
tubespervisitid1.reset_index(drop=True, inplace=True)
display(tubespervisitid1)
display(Markdown("**%d** tubes for VisitID." % tubespervisitid1["Tubes"].sum()))

display(Markdown("#### ThawCycle field"))
tubesperthawcycle1 = pd.DataFrame(df_frac1.groupby("ThawCycle")["ThawCycle"].count())
tubesperthawcycle1.loc[:, "Tubes"] = tubesperthawcycle1["ThawCycle"]
tubesperthawcycle1.loc[:, "ThawCycle"] = tubesperthawcycle1.index
tubesperthawcycle1.reset_index(drop=True, inplace=True)
display(tubesperthawcycle1)
display(Markdown("**%d** tubes for ThawCycle." % tubesperthawcycle1["Tubes"].sum()))

display(Markdown("#### Sample Source field"))
display(Markdown("* **%d** empty Sample Source\n* **%d** unique Sample Source" % \
                 (len(df_frac1[df_frac1["Sample Source"].isnull()]), \
                  len(df_frac1["Sample Source"].unique()))))

display(Markdown("#### BatchID field"))
tubesperbatchid1 = pd.DataFrame(df_frac1.groupby("BatchID")["BatchID"].count())
tubesperbatchid1.loc[:, "Tubes"] = tubesperbatchid1["BatchID"]
tubesperbatchid1.loc[:, "BatchID"] = tubesperbatchid1.index
tubesperbatchid1.reset_index(drop=True, inplace=True)
display(tubesperbatchid1)
display(Markdown("**%d** tubes for ThawCycle." % tubesperbatchid1["Tubes"].sum()))

display(Markdown("#### Sample Type field"))
tubespersampletype1 = pd.DataFrame(df_frac1.groupby("Sample Type")["Sample Type"].count())
tubespersampletype1.loc[:, "Tubes"] = tubespersampletype1["Sample Type"]
tubespersampletype1.loc[:, "Sample Type"] = tubespersampletype1.index
tubespersampletype1.reset_index(drop=True, inplace=True)
display(tubespersampletype1)
display(Markdown("**%d** tubes for Sample Type." % tubespersampletype1["Tubes"].sum()))

display(Markdown("### Look on Fraction 2"))

display(Markdown("#### ParentID field"))
display(Markdown("* %d empty ParentID\n* %d unique ParentID" % \
                 (len(df_frac2[df_frac2["ParentID"].isnull()]), \
                  len(df_frac2["ParentID"].unique()))))

display(Markdown("#### Position field"))
tubesperposition2 = pd.DataFrame(df_frac2.groupby("Position")["Position"].count())
tubesperposition2.loc[:, "Tubes"] = tubesperposition2["Position"]
tubesperposition2.loc[:, "Position"] = tubesperposition2.index
tubesperposition2.reset_index(drop=True, inplace=True)
display(tubesperposition2)
display(Markdown("**%d** tubes for Position." % tubesperposition2["Tubes"].sum()))

display(Markdown("#### CreationDate field"))
tubespercreation2 = pd.DataFrame(df_frac2.groupby("CreationDate")["CreationDate"].count())
tubespercreation2.loc[:, "Tubes"] = tubespercreation2["CreationDate"]
tubespercreation2.loc[:, "CreationDate"] = tubespercreation2.index
tubespercreation2.reset_index(drop=True, inplace=True)
display(tubespercreation2)
display(Markdown("**%d** tubes for CreationDate." % tubespercreation2["Tubes"].sum()))

display(Markdown("#### UpdateDate field"))
tubesperupdate2 = pd.DataFrame(df_frac2.groupby("UpdateDate")["UpdateDate"].count())
tubesperupdate2.loc[:, "Tubes"] = tubesperupdate2["UpdateDate"]
tubesperupdate2.loc[:, "UpdateDate"] = tubesperupdate2.index
tubesperupdate2.reset_index(drop=True, inplace=True)
display(tubesperupdate2)
display(Markdown("**%d** tubes for UpdateDate." % tubesperupdate2["Tubes"].sum()))

display(Markdown("#### AliquotID field"))
tubesperaliquotid2 = pd.DataFrame(df_frac2.groupby("AliquotID")["AliquotID"].count())
tubesperaliquotid2.loc[:, "Tubes"] = tubesperaliquotid2["AliquotID"]
tubesperaliquotid2.loc[:, "AliquotID"] = tubesperaliquotid2.index
tubesperaliquotid2.reset_index(drop=True, inplace=True)
display(tubesperaliquotid2)
display(Markdown("**%d** tubes for AliquotID." % tubesperaliquotid2["Tubes"].sum()))

display(Markdown("#### BoxType field"))
tubesperboxtype2 = pd.DataFrame(df_frac2.groupby("BoxType")["BoxType"].count())
tubesperboxtype2.loc[:, "Tubes"] = tubesperboxtype2["BoxType"]
tubesperboxtype2.loc[:, "BoxType"] = tubesperboxtype2.index
tubesperboxtype2.reset_index(drop=True, inplace=True)
display(tubesperboxtype2)
display(Markdown("**%d** tubes for BoxType." % tubesperboxtype2["Tubes"].sum()))

display(Markdown("#### VisitID field"))
tubespervisitid2 = pd.DataFrame(df_frac2.groupby("VisitID")["VisitID"].count())
tubespervisitid2.loc[:, "Tubes"] = tubespervisitid2["VisitID"]
tubespervisitid2.loc[:, "VisitID"] = tubespervisitid2.index
tubespervisitid2.reset_index(drop=True, inplace=True)
display(tubespervisitid2)
display(Markdown("**%d** tubes for VisitID." % tubespervisitid2["Tubes"].sum()))

display(Markdown("#### ThawCycle field"))
tubesperthawcycle2 = pd.DataFrame(df_frac2.groupby("ThawCycle")["ThawCycle"].count())
tubesperthawcycle2.loc[:, "Tubes"] = tubesperthawcycle2["ThawCycle"]
tubesperthawcycle2.loc[:, "ThawCycle"] = tubesperthawcycle2.index
tubesperthawcycle2.reset_index(drop=True, inplace=True)
display(tubesperthawcycle2)
display(Markdown("**%d** tubes for ThawCycle." % tubesperthawcycle2["Tubes"].sum()))

display(Markdown("#### Sample Source field"))
display(Markdown("* **%d** empty Sample Source\n* **%d** unique Sample Source" % \
                 (len(df_frac2[df_frac2["Sample Source"].isnull()]), \
                  len(df_frac2["Sample Source"].unique()))))

display(Markdown("#### BatchID field"))
tubesperbatchid2 = pd.DataFrame(df_frac2.groupby("BatchID")["BatchID"].count())
tubesperbatchid2.loc[:, "Tubes"] = tubesperbatchid2["BatchID"]
tubesperbatchid2.loc[:, "BatchID"] = tubesperbatchid2.index
tubesperbatchid2.reset_index(drop=True, inplace=True)
display(tubesperbatchid2)
display(Markdown("**%d** tubes for ThawCycle." % tubesperbatchid2["Tubes"].sum()))

display(Markdown("#### Sample Type field"))
tubespersampletype2 = pd.DataFrame(df_frac2.groupby("Sample Type")["Sample Type"].count())
tubespersampletype2.loc[:, "Tubes"] = tubespersampletype2["Sample Type"]
tubespersampletype2.loc[:, "Sample Type"] = tubespersampletype2.index
tubespersampletype2.reset_index(drop=True, inplace=True)
display(tubespersampletype2)
display(Markdown("**%d** tubes for Sample Type." % tubespersampletype2["Tubes"].sum()))

### Look on Fraction 1

#### ParentID field

* 0 empty ParentID
* 10000 unique ParentID

#### Position field

,Position,Tubes
0,A / 1,110
1,A / 10,110
2,A / 11,100
3,A / 12,100
4,A / 2,110
5,A / 3,110
6,A / 4,110
7,A / 5,110
8,A / 6,110
9,A / 7,110


...

,BatchID,Tubes
0,A,6060
1,B,3940


**10000** tubes for ThawCycle.

#### Sample Type field

,Sample Type,Tubes
0,Fraction2,10000


**10000** tubes for Sample Type.

## Checking some fields per fraction

We need to check if the number of unique BARCODE is coherent between each fraction:

In [17]:
display(Markdown("#### BARCODE field"))
tubesperbarcode = compare_two_columns(df, "Sample Type", "BARCODE", \
                                      method="nunique")

display(Markdown("**{}** unique BARCODE for Sample Type.".\
                 format(tubesperbarcode["Nb_BARCODE"].sum())))

display(Markdown("#### Freezer field"))
tubesperfreezer = compare_two_columns(df, "Freezer", "BARCODE", method="nunique")

#### BARCODE field

,Sample Type,Nb_BARCODE
0,Fraction1,10000
1,Fraction2,10000


KeyError: 'Nb_BARCODE'

Check if RFID are assigned to more than 2 BARCODE.

In [ ]:
barcodeperrfid = compare_two_columns(df, "ParentID", "BARCODE", \
                                     method="nunique", show=False)

if len(barcodeperrfid[barcodeperrfid["BARCODE"] == 0]) > 0:
    nobarcode = barcodeperrfid[barcodeperrfid["BARCODE"] == 0]
    display(Markdown("**{}** ParentID not assigned to a BARCODE.".\
                     format(len(nobarcode))))

if len(barcodeperrfid[barcodeperrfid["BARCODE"] == 1]) > 0:
    onebarcode = barcodeperrfid[barcodeperrfid["BARCODE"] == 1]
    display(Markdown("**{}** ParentID assigned to 1 BARCODE.".\
                     format(len(onebarcode))))

if len(barcodeperrfid[barcodeperrfid["BARCODE"] == 2]) > 0:
    twobarcode = barcodeperrfid[barcodeperrfid["BARCODE"] == 2]
    display(Markdown("**{}** ParentID assigned to 2 BARCODE.".\
                     format(len(twobarcode))))

if len(barcodeperrfid[barcodeperrfid["BARCODE"] > 2 ]) > 0:
    morebarcode = barcodeperrfid[barcodeperrfid["BARCODE"] > 2]
    display(Markdown("**{}** ParentID assigned to more than 2 BARCODE:".\
                     format(len(morebarcode))))